In [56]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import glob
import string
import os
import codecs

## Project Gutenberg

In [37]:
BASE_BOOK_URL = 'https://www.gutenberg.org/browse/scores/top'

html = requests.get(BASE_BOOK_URL).text
soup = BeautifulSoup(html)

unq_code = {}
for s in soup.findAll('li'):
    url = s.a['href']
    if 'ebooks' in url:
        unq_code[int(url.split('/')[-1])] = s.a.text

In [38]:
BOOK_TXT_BASE = 'https://www.gutenberg.org/files/'
book_urls = []
for code in unq_code:
    book_urls.append(os.path.join(BOOK_TXT_BASE, f'{code}/{code}-0.txt'))

In [40]:
for b in book_urls:
    name = b.split('/')[-2]
    html = requests.get(b).text
    with codecs.open(f'data/books/{name}.txt', 'w', 'utf-8') as infile:
        infile.write(html)

## GradeSaver Glossary

In [67]:
BASE_GLOSS_URL = 'https://www.gradesaver.com/'
TERMINAL = '/study-guide/glossary-of-terms'

def punctuations(data_str):
    data_str = data_str.replace("'s", "")
    for x in data_str.lower():
        if x in string.punctuation: 
            data_str = data_str.replace(x, "")
    return data_str

for book in glob.glob('data/books/*'):
    code = book.split('/')[-1].split('.')[0]
    try:
        bookname = unq_code[int(code)]
        bookname = bookname.split(' by ')[0].lower()
        bookname = punctuations(bookname)
        bookname = bookname.replace(" ", "-")
        html = requests.get(BASE_GLOSS_URL+bookname+TERMINAL).text
        soup = BeautifulSoup(html)
        tt = []
        for term in soup.findAll("section", {"class": "linkTarget"}):
            tt.append([term.h2.text.lower().strip(), term.p.text.lower().strip()])
        if len(tt):
            print (f'Done: {bookname}')
            data = pd.DataFrame(tt, columns=['word', 'def'])
            data.to_csv(f'data/ground_truth/{code}.csv', sep='\t', encoding='utf-8', index=False)
        else:
            print (f'Skipped: {bookname}')
    except Exception as e: print (e)

Skipped: the-awakening-and-selected-short-stories
Done: the-adventures-of-sherlock-holmes
Skipped: songs-of-innocence-and-songs-of-experience
Skipped: ang-filibusterismo-karugtóng-ng-noli-me-tangere
Skipped: the-£1000000-banknote-and-other-new-stories
Skipped: ulysses
Skipped: the-call-of-the-wild
Done: pygmalion
Done: the-souls-of-black-folk
Skipped: also-sprach-zarathustra-english
Skipped: moby-dick-or-the-whale
Skipped: ukridge
Skipped: the-life-and-adventures-of-robinson-crusoe
Done: the-odyssey
invalid literal for int() with base 10: 'HP7'
Skipped: frankenstein-or-the-modern-prometheus
Skipped: the-interesting-narrative-of-the-life-of-olaudah-equiano-or-gustavus-vassa-the-african
Skipped: anna-karenina
Skipped: essays-of-michel-de-montaigne-—-complete
Skipped: metamorphosis
Skipped: great-expectations
Done: the-scarlet-letter
Done: little-women
Skipped: the-prophet
Skipped: a-tale-of-two-cities
Done: common-sense
Skipped: treasure-island
Skipped: dubliners
Skipped: the-modern-trav